# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [1]:
from datasets import load_dataset, disable_caching
from transformers import (
    AutoTokenizer,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

from collections import defaultdict, Counter
from tqdm.auto import tqdm
import numpy as np

import os
import re
import string

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


Set cache directory.

In [2]:
CACHE_DIR='/mount/arbeitsdaten31/studenten1/linku/.cache'
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache
env: HF_MODULES_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache
env: HF_DATASETS_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache


### arguments.py

args_input.

In [3]:
args_input_ALstrategy = 'MarginSampling'
# args_input_initseed = 100 # 1000
args_input_quota = 200 # 200
args_input_batch = 50 # 50
args_input_dataset_name = 'DROP'
args_input_expe_round = 2
args_input_model_batch = 8 # already add in arguments.py
args_input_max_length = None
args_input_learning_rate = 3e-5
args_input_model = 'RoBERTa'

stride = 128

in main.py

In [4]:
MAX_LENGTH = args_input_max_length
NUM_QUERY = args_input_batch
# NUM_INIT_LB = args_input_initseed
ITERATION = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

MODEL_NAME = args_input_model
LEARNING_RATE = args_input_learning_rate
EXPE_ROUND = args_input_expe_round
MODEL_BATCH = args_input_model_batch
NUM_TRAIN_EPOCH = 3


In [5]:
model_dir = '/mount/arbeitsdaten31/studenten1/linku/models'
pretrain_model_dir = '/mount/arbeitsdaten31/studenten1/linku/pretrain_models' + '/' + 'RoBERTa_SQuAD_full_dataset'

### load dataset

In [6]:
disable_caching()

In [7]:
def load_dataset_mrqa(d):
	'''
	return train_set, val_set
	'''
	data = load_dataset("mrqa", cache_dir=CACHE_DIR)
	if d == 'squad':
		# the first to 86588th in train set
		# the first to 10507th in val set		
		return data['train'].select(range(86588)), data['validation'].select(range(10507))
	elif d == 'newsqa':
		# the 86589th to 160748th in train set
		# the 10508th to 14719th in val set		
		return data['train'].select(range(86588, 160748)), data['validation'].select(range(10507, 14719))
	elif d == 'searchqa':
		# the 222437th to 339820th in train set
		# the 22505th to 39484th in val set
		return data['train'].select(range(222436, 339820)), data['validation'].select(range(22504, 39484))
	elif d == 'bioasq':
		# the first to the 1504th in the test set
		sub = data['test'].select(range(1504))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val))
	elif d == 'textbookqa':
		# the 8131st to 9633rd
		sub = data['test'].select(range(8130, 9633))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val)) 
	elif d == 'drop': # Discrete Reasoning Over Paragraphs
		# the 1505th to 3007th in test set
		sub = data['test'].select(range(1504, 3007))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val))

In [8]:
# TODO: function different!!!

# train_data, val_data = load_dataset_mrqa(DATA_NAME.lower())
# train_data, val_data = load_dataset_mrqa('squad')
train_data, val_data = load_dataset_mrqa('drop')

Found cached dataset mrqa (/mount/arbeitsdaten31/studenten1/linku/.cache/mrqa/plain_text/1.1.0/1f2cf5ac32b43f864e6f91d384057a16b69b7d13ba9bcaa200ac277c90938d19)


  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
# train_data = train_data.select(range(1000))
# val_data = val_data.select(range(100))

Next we will preprocess the dataset (training and evaluation data).

In [10]:
def preprocess_training_examples_lowRes(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    detected_answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = detected_answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["qid"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
def preprocess_training_features_lowRes(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["qid"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
def preprocess_validation_examples_lowRes(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["qid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [13]:
# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [14]:
# TODO: The data name different!!!

# preprocess data
train_dataset = train_data.map(
    preprocess_training_examples_lowRes,
    batched=True,
    remove_columns=train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
train_features = train_data.map(
    preprocess_training_features_lowRes,
    batched=True,
    remove_columns=train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
val_dataset = val_data.map(
    preprocess_validation_examples_lowRes,
    batched=True,
    remove_columns=val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
val_features = val_data.map(
    preprocess_validation_examples_lowRes,
    batched=True,
    remove_columns=val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)

Map:   0%|          | 0/1353 [00:00<?, ? examples/s]

Map:   0%|          | 0/1353 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [15]:
train_dataset.set_format("torch")
train_features.set_format("torch")
val_dataset = val_dataset.remove_columns(["offset_mapping"])
val_dataset.set_format("torch")
val_features.set_format("torch")

## model_lowRes.py

In [16]:
strategy_model_dir = model_dir + '/lowRes_' + str(args_input_quota) + '_' + STRATEGY_NAME + '_' + 'RoBERTa' +  '_' + DATA_NAME

In [17]:
def to_train_lowRes(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler, record_loss=False):
	print('Training was performed using {} query data, i.e. {} data.'.format(NUM_QUERY, len(train_dataloader.dataset)))
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss
			loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()

		if record_loss:
			print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

	model_to_save = model.module if hasattr(model, 'module') else model 
	model_to_save.save_pretrained(strategy_model_dir)
	print('TRAIN done!')

In [18]:
def get_pred_lowRes(dataloader, device, features, examples):
    model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    return compute_metrics(start_logits, end_logits, features, examples)

In [19]:
def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = dict()
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["qid"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers[example_id] = best_answer["text"]
        else:
            predicted_answers[example_id] = ""

    theoretical_answers = dict()
    for ex in examples: theoretical_answers[ex["qid"]] = ex["answers"]
    return evaluate(theoretical_answers, predicted_answers)

In [20]:
def get_prob_lowRes(dataloader, device, features, examples, rd=0):
    if rd == 1:
        model = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
    else:
        model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)

    model.eval()
    start_logits = []
    end_logits = []

    for batch in tqdm(dataloader, desc="Evaluating_prob"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    prob_dict = {}
    example_to_features = defaultdict(list)
    max_answer_length = 30
    n_best = 20
    
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    for example in tqdm(examples):
        example_id = example["qid"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])
        
            if len(answers) > 1:
                prob_dict[feature_index] = softmax(answers)
            elif example_to_features[example_id] != []:
                prob_dict[feature_index] = np.array([0])
    
    return prob_dict

In [21]:
def get_prob_dropout_lowRes(dataloader, device, features, examples, n_drop=10, rd=0):
    if rd == 1:
        model = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
    else:
        model = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)
    
    model.train()
    prob_dict = {}
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["qid"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            if 1 < len(answers) < 200: # pad to same numbers of possible answers
                zero_list = [0] * (200 - len(answers))
                answers.extend(zero_list)
            elif len(answers) >= 200:
                answers = answers[:200]

            if len(answers) > 1:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = softmax(answers)
                else:
                    prob_dict[example_to_features[example_id][0]] += softmax(answers)
            elif example_to_features[example_id] != []:
                if example_to_features[example_id][0] not in prob_dict:
                    prob_dict[example_to_features[example_id][0]] = np.array([0])   

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop

    return prob_dict

## utils.py

In [22]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data

In [23]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    softmax_num = np.exp(x) / np.sum(np.exp(x), axis=0)
    return np.round(softmax_num, decimals=4)

In [24]:
def get_aubc(quota, bsize, resseq):
	# it is equal to use np.trapz for calculation
	ressum = 0.0
	if quota % bsize == 0:
		for i in range(len(resseq)-1):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2

	else:
		for i in range(len(resseq)-2):
			ressum = ressum + (resseq[i+1] + resseq[i]) * bsize / 2
		k = quota % bsize
		ressum = ressum + ((resseq[-1] + resseq[-2]) * k / 2)
	ressum = round(ressum / quota,3)
	
	return ressum

In [25]:
def get_mean_stddev(datax):
	return round(np.mean(datax),4),round(np.std(datax),4)

In [26]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [27]:
def evaluate(answers, predictions, skip_no_answer=False):
    f1 = exact_match = total = 0
    for qid, ground_truths in answers.items():
        if qid not in predictions:
            if not skip_no_answer:
                message = 'Unanswered question %s will receive score 0.' % qid
                print(message)
                total += 1
            continue
        total += 1
        prediction = predictions[qid]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

## Query

In [28]:
# def random_sampling_query(labeled_idxs, n):
#     print('Random querying starts!')
#     return np.random.choice(np.where(labeled_idxs==0)[0], n, replace=False)

In [29]:
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd=0):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    print('Margin querying starts!')
    print('Query {} data from {} unlabeled training data.'.format(n, len(unlabeled_data)))
    prob_dict = get_prob_lowRes(unlabeled_dataloader, device, unlabeled_features, examples, rd)
    print('Got probability!')
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties_dict[idx] = sort_probs[0] - sort_probs[1]
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

# main.py

### seed and device

In [30]:
# get the number of extra data after preprocessing
extra = min(NUM_QUERY, len(train_dataset) - len(train_data))

SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [31]:
all_acc = []
acq_time = []

In [32]:
## generate initial labeled pool
n_pool = len(train_dataset)
labeled_idxs = np.zeros(n_pool, dtype=bool)

## record acc performance 
acc = np.zeros(ITERATION) # quota/batch runs
acc_em = np.zeros(ITERATION)

## load the selected train data to DataLoader
eval_dataloader = DataLoader(
    val_dataset, 
    collate_fn=default_data_collator, 
    batch_size=MODEL_BATCH
)

In [33]:
i = 1
## use total_query (NUM_QUERY + extra) to query instead of just NUM_QUERY
total_query = NUM_QUERY + extra

## query
q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, total_query, i)

# print('Time spent for querying:', (datetime.datetime.now() - time))
# time = datetime.datetime.now()

Margin querying starts!
Query 100 data from 1415 unlabeled training data.


Evaluating_prob:   0%|          | 0/177 [00:00<?, ?it/s]

  0%|          | 0/1353 [00:00<?, ?it/s]

Got probability!


In [43]:
## update
    
## goal of total query data: NUM_QUERY * i
num_set_query_i = NUM_QUERY * i

# difference_i = 0
# num_set_ex_id_i = 0
labeled_idxs[q_idxs[:NUM_QUERY]] = True
run_i_labeled_idxs = np.arange(n_pool)[labeled_idxs]

run_i_samples = train_features.select(indices=run_i_labeled_idxs)
num_set_ex_id_i = len(set(run_i_samples['example_id']))

difference_i = num_set_query_i - num_set_ex_id_i

if difference_i:
    labeled_idxs[q_idxs[NUM_QUERY:(NUM_QUERY + difference_i)]] = True
    run_i_labeled_idxs = np.arange(n_pool)[labeled_idxs]

In [44]:
# run_i_samples = train_features.select(indices=run_i_labeled_idxs)
# num_set_ex_id_i = len(set(run_i_samples['example_id']))

# num_set_query_i - num_set_ex_id_i

0

In [47]:
train_dataloader_i = DataLoader(
    train_dataset.select(indices=run_i_labeled_idxs),
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=MODEL_BATCH,
)

num_update_steps_per_epoch_i = len(train_dataloader_i)
num_training_steps_i = NUM_TRAIN_EPOCH * num_update_steps_per_epoch_i

if i == 1:
    print('Use pretrain model in iteration ', i)
    model_i = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
else:
    print('Use strategy model in iteration ', i)
    model_i = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)

optimizer_i = AdamW(model_i.parameters(), lr=LEARNING_RATE)

lr_scheduler_i = get_scheduler(
    "linear",
    optimizer=optimizer_i,
    num_warmup_steps=0,
    num_training_steps=num_training_steps_i,
)

## train
to_train_lowRes(NUM_TRAIN_EPOCH, train_dataloader_i, device, model_i, optimizer_i, lr_scheduler_i)

Use pretrain model in iteration  1
Training was performed using 50 query data, i.e. 52 data.


Training:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/7 [00:00<?, ?it/s]

Training:   0%|          | 0/7 [00:00<?, ?it/s]

TRAIN done!


In [48]:
# iteration i accuracy
print('iter_{} get_pred!'.format(i))
acc_scores_i = get_pred_lowRes(eval_dataloader, device, val_features, val_data)
acc[i] = acc_scores_i['f1']
acc_em[i] = acc_scores_i['exact_match']
print('testing accuracy {}'.format(acc[i]))
print('testing accuracy em {}'.format(acc_em[i]))

iter_1 get_pred!


Evaluating_pred:   0%|          | 0/19 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/150 [00:00<?, ?it/s]

testing accuracy 49.63968253968253
testing accuracy em 40.666666666666664


In [49]:
i = 2
## use total_query (NUM_QUERY + extra) to query instead of just NUM_QUERY
total_query = NUM_QUERY + extra

## query
q_idxs = margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, train_data, device, total_query, i)

# print('Time spent for querying:', (datetime.datetime.now() - time))
# time = datetime.datetime.now()

## update
    
## goal of total query data: NUM_QUERY * i
num_set_query_i = NUM_QUERY * i

# difference_i = 0
# num_set_ex_id_i = 0
labeled_idxs[q_idxs[:NUM_QUERY]] = True
run_i_labeled_idxs = np.arange(n_pool)[labeled_idxs]

run_i_samples = train_features.select(indices=run_i_labeled_idxs)
num_set_ex_id_i = len(set(run_i_samples['example_id']))

difference_i = num_set_query_i - num_set_ex_id_i

if difference_i:
    labeled_idxs[q_idxs[NUM_QUERY:(NUM_QUERY + difference_i)]] = True
    run_i_labeled_idxs = np.arange(n_pool)[labeled_idxs]

Margin querying starts!
Query 100 data from 1363 unlabeled training data.


Evaluating_prob:   0%|          | 0/171 [00:00<?, ?it/s]

  0%|          | 0/1353 [00:00<?, ?it/s]

Got probability!


In [50]:
train_dataloader_i = DataLoader(
    train_dataset.select(indices=run_i_labeled_idxs),
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=MODEL_BATCH,
)

num_update_steps_per_epoch_i = len(train_dataloader_i)
num_training_steps_i = NUM_TRAIN_EPOCH * num_update_steps_per_epoch_i

if i == 1:
    print('Use pretrain model in iteration ', i)
    model_i = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
else:
    print('Use strategy model in iteration ', i)
    model_i = AutoModelForQuestionAnswering.from_pretrained(strategy_model_dir).to(device)

optimizer_i = AdamW(model_i.parameters(), lr=LEARNING_RATE)

lr_scheduler_i = get_scheduler(
    "linear",
    optimizer=optimizer_i,
    num_warmup_steps=0,
    num_training_steps=num_training_steps_i,
)

## train
to_train_lowRes(NUM_TRAIN_EPOCH, train_dataloader_i, device, model_i, optimizer_i, lr_scheduler_i)

## iteration i accuracy
print('iter_{} get_pred!'.format(i))
acc_scores_i = get_pred_lowRes(eval_dataloader, device, val_features, val_data)
acc[i] = acc_scores_i['f1']
acc_em[i] = acc_scores_i['exact_match']
print('testing accuracy {}'.format(acc[i]))
print('testing accuracy em {}'.format(acc_em[i]))

Use strategy model in iteration  2
Training was performed using 50 query data, i.e. 106 data.


Training:   0%|          | 0/14 [00:00<?, ?it/s]

Training:   0%|          | 0/14 [00:00<?, ?it/s]

Training:   0%|          | 0/14 [00:00<?, ?it/s]

TRAIN done!
iter_2 get_pred!


Evaluating_pred:   0%|          | 0/19 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/150 [00:00<?, ?it/s]

testing accuracy 56.44481074481076
testing accuracy em 48.0
